# Iterative Solvers

## Smoothers
Observations from the previous notebook:
- slow convergence
- smooth error components are reduced very slowly
- high frequency error components are damped more efficiently 

Let's go back to our model problem, the Poisson equation. In 1D:

\begin{align}
- \partial_x^2 u (x) &= 0\text{ on }\Omega = (0, 1),\\
u(0) &= u(1) = 0
\end{align}

We will start with a 0 right hand side and boundary conditions. Note that in this case the solution will be zero. Nevertheless, looking at the results will be useful for seeing why Jacobi reduces high-frequency components efficiently.

We discretise on a uniform grid of mesh size h = 1/(N+1). The grid points are $x_j := jh$, with $j = 1, . . . , N$. As before we compute values at grid points $u_j  = u(x_j)$ and use a second-order accurate approximation:
$$\partial_x^2 u(x) \approx \frac{u(x+h)-2u(x)+u(x-h)}{h^2}$$

Recall that the Jacobi relaxation will give the scheme:
$u_j^{(k +1)} = \frac{1}{2}(u_{j+1}^{(k)} + u_{j−1}^{(k)})$
In this case, we know that the exact solution will be zero, so the error is just
$$e_j^{(k)} = 0 - u_j^{(k)}$$

### Eigenvalues and -vectors of $A$ (see exercise)
- eigenvalues: $$\lambda_k = \frac{4}{h^2} \sin^2\bigl(\frac{k \pi}{2n}\bigr) = \frac{4}{h^2} \sin^2\bigl(\frac{k \pi h}{2}\bigr)$$
- eigenvectors: $$v^{(k)} = \bigl( \sin(k \pi j / n) \bigr)_{j=1,\dots,n-1}$$
    - both for $k=1,\dots, N$

### Eigenvalues and -vectors of $M$
Recall that the iteration matrix for a Jacobi iteration was:
$$M = I - D_A^{-1} A = I - \frac {h^2}{2} A$$

Using the computation for the eigenvalues of $A$ we can now compute the eigenvalues of $M$. We know that
$$\mathop{diag}\left({A}\right)^{-1} = \frac{h^2}{2}I.$$
It can be easily shown that the following is true for every $k=1,\dots,N-1$:
$$Mv_k = \left(I-\frac{h^2}{2}A\right)v_k = \left(1-\frac{h^2}{2}\lambda_k\right)v_k =: \mu_kv_k.$$

Hence, the eigenvectors of $M$ are $v_k$ and the eigenvalues are
$$ \mu_k = 1-\frac{h^2}{2}\lambda_k = 1-\frac{h^2}{2}\frac{4}{h^2}\sin^2\left(\frac{k\pi h}{2}\right) = 1-2\sin^2\left(\frac{k\pi h}{2}\right).$$

As derived in the previous notebook, the (point-wise) error in the $n$-th iteration is
$$
  e^{(n)} = M^ne^{(0)}.
$$
We may decompose any error into a linear combination of eigenvectors $v_k$, as they are a basis of $\mathbb{R}^{N-1}$. That is,
$$
  e^{(0)} = \sum_{k=1}^{N-1}\alpha_k v_k.
$$
Inserting this into the error equation yields
$$
  e^{(n)} = M^n\left(\sum_{k=1}^{N-1}\alpha_k v_k\right) = \sum_{k=1}^{N-1}\alpha_k M^nv_k = \sum_{k=1}^{N-1}\alpha_k \lambda_k^nv_k.
$$
If we take the norm on both sides and use the triangle inequality we obtain
\begin{equation}
  \|e^{(n)}\| = \left\|\sum_{k=1}^{N-1}\alpha_k \lambda_k^nv_k\right\| \leq \sum_{k=1}^{N-1}|\alpha_k| |\lambda_k|^n\left\|v_k\right\|.
\end{equation}

This leads to the following interpretation: As the eigenvectors follow a sine function, we may think
of them as waves with a single frequency. The error can be decomposed into frequency components, similar to a Fourier analysis.
The eigenvalues then give an upper bound on the error reduction of each frequency component. So even if we are unaware of the error (because
the true solution is unknown in general) and of the $\alpha_k$ values, we still know that the error in the $k$-th frequency component 
is reduced by the factor $|\lambda_k|$ in one iteration.

When seeing a minimax problem, one can think of it in terms of a protagonist and an antagonist.
Here, the protagonist (you) tries to minimise the eigenvalues of $M$ such that the error is reduced fastest
in each iteration. Then, the antagonist (Murphy) picks the largest eigenvalue, i.e. the one leading to the worst error reduction.
(In a sense, Murphy picks the worst of all the things that can go wrong.)
The good thing about a minimax problem is that you get a guaranteed bound that holds even in the worst case.

Coming to the problem at hand: As the eigenvalues are monotonically decreasing, it is sufficient to take the maximum
over the first and the last eigenvalue in the interval, i.e.
$$
\omega = \mathop{argmin}_{\omega'}\max\left\{|\lambda_{N/2}(\omega')|, |\lambda_{N}(\omega')|\right\}
= \mathop{argmin}_{\omega'}\max\left\{|1-\omega'|, |1-2\omega'|\right\}.
$$
The extreme points of the maximum function must be at at one of the kinks or intersection points of $|1-\omega'|$ and $|1-2\omega'|$, because otherwise
one could follow the slope to a smaller or larger point (hint: create a plot of $\max\left\{|\lambda_{N/2}(\omega')|, |\lambda_{N}(\omega')|\right\}$).
Here, the only relevant case (for finding the minimum) is 
$$
  \lambda_{N/2} = 1-\omega' = -\left(1-2\omega'\right) = -\lambda_{N},
$$
which leads to $\omega = 2/3$. With $\omega = 2/3$, the maximum absolute eigenvalue for the high frequencies is $1/3$. Hence,
all errors in high frequencies ($k \geq N/2$) are guaranteed to be three times lower after one iteration.


### Observation
- The <b>high</b> frequency part (with respect to the underlying grid) is reduced quite quickly.

- The <b>low</b> frequency part (w.r.t. the grid) decreases only very slowly; actually the slower, the finer the grid is.

- Smoothers good for reducing high-frequency errors
- Otherwise very slow convergence
- Gauss-Seidel and SOR, usually slightly faster than Jacobi